# Sample answer to Activity 1

*Last updated by Christian Cahig on 2025-10-06*

## Imports

In [1]:
import math as mt

import scipy.optimize as spo

## Scenario

You are a power systems engineer tasked to analyze two customer loads and served at a common bus.
The bus is at the receiving end of a feeder line extending from the utility system.

One of the loads is estimated to be 400 kilowatts at a lagging power factor of 0.85,
while the other is rated 300 kilovars at a lagging power factor of 0.75.

Records show that the feeder line impedance is rated 1.00 + j2.50 ohms.
If it is reasonable to assume that the utility system's RMS voltage hovers around 13.8 kilovolts,
at what RMS voltage are the loads served?

You are to approach this as a root-finding problem,
and so must first derive a function $f\!\left(V\right)$
whose root corresponds to the desired quantity.

## Modelling

The scenario describes a radial topology:
power flows from where the feeder line connects to the utility system,
through the feeder line,
and to the common bus.
See [./fig-01.png](./fig-01.png).

Without any more information,
we assume we are dealing with steady-state sinusoidal signals.
This allows us to use frequency-domain analysis via phasor representation.
Furthermore, we can set the voltage phasor at the common bus to have the reference phase angle,
and the voltage phasor "upstream" we can set to have some phase angle $\theta$.
As such, we can arrive at the following relation:

$$
13.8 \angle \theta = V \angle 0 + (1 + j2.5) I_{\text{load}},
$$

where $V$ is the magnitude (in kilovolts) of the voltage phasor at the common bus,
and $I_{\text{load}}$ is the total current (in kiloamperes) through the feeder line,
as induced by the loads.
Moreover,

$$
\left(V \angle 0\right) \operatorname{conj}\!\left( I_{\text{load}} \right)
=
\frac{0.4}{0.85} \angle\arccos\!\left(0.85\right)
+
\frac{0.3}{\sin\!\left(\arccos\!\left(0.75\right)\right)} \angle\arccos\!\left(0.75\right).
$$

It is then straightforward to arrive at

$$
13.8^{2} = \left(V + \frac{2.109912364}{V}\right)^{2} + \left(\frac{1.302522329}{V}\right)^{2},
$$

and so a plausible univariate function to root-find is

$$
f\!\left(V\right) = 
\left(V + \frac{2.109912364}{V}\right)^{2} + \left(\frac{1.302522329}{V}\right)^{2}
- 13.8^{2}.
$$

## Bisection

Construct a Python function `resid_from_x()` that implements $f\!\left(V\right)$.
The function takes a single positional argument `x`,
representing $V$.

In [2]:
def resid_from_x(x):
    return (x + (2.109912364 / x))**2 + (1.302522329 / x)**2 - 13.8**2

Use bisection to find a root of $f\!\left(V\right)$.
Use an iteration budget of 750 and a tolerance of $10^{-7}$.

Define the following variables.
- `XL`, to store the lower end of the search interval
- `XU`, to store the upper end of the search interval
- `MAX_ITERS`, to store the iteration budget
- `X_TOL` to store the tolerance

Run
[`scipy.optimize.bisect()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.bisect.html)
such that you get the (approximate) root as well as information concerning the run.
Store the root and the information in variables `p_bs` and `p_bs_info`, respectively.
Make sure to verify if the returned root is indeed a root.

In [3]:
XL, XU = 5, 14
MAX_ITERS = 750
X_TOL = 1e-7

p_bs, p_bs_info = spo.bisect(
    resid_from_x, XL, XU,
    xtol=X_TOL, maxiter=MAX_ITERS,
    full_output=True, disp=False
)

## Results

Do not modify nor remove the following code cell.

In [4]:
print(f"Finding a root in the interval [{XL}, {XU}]:")
print(p_bs_info)
print(f"Residual value: {resid_from_x(p_bs)}")

Finding a root in the interval [5, 14]:
      converged: True
           flag: converged
 function_calls: 29
     iterations: 27
           root: 13.645041354000568
         method: bisect
Residual value: 4.0078435858958983e-07


In [5]:
print(f"The RMS voltage at the common bus is {p_bs} kilovolts.")

The RMS voltage at the common bus is 13.645041354000568 kilovolts.


## Some things to consider

- Due to the radial topology,
it is reasonable to expect
the RMS voltage at the common bus to be smaller in magnitude
than that at which the feeder is coupled to the utility system.

- It is practically unreasonable to set the lower limit of the search interval to 0
mainly because that value is something that is to be avoided in practice.
Although, mathematically, the search interval may range from a negative to positive value.

- Try to make the convergence criterion stricter
and verify if the run indeed takes more iterations.

- What about a different $f\!\left(V\right)$?

- Does $f\!\left(V\right)$ have other roots?